In [1]:
import numpy as np
import pandas as pd

In [2]:
file = 'global_data_4_0.5'
file_split = np.array(file.split('_'))[-2:].astype(float)

time_bin  = file_split[-1]
H_bin  = file_split[-2]

time_bins = np.arange(0,24,time_bin)
H_bins = np.arange(90,130,H_bin)

shape = len(time_bins) * len(H_bins)

file_path = f'../../Output/{file}.csv'
supermag_path = '../../Index/omni_global.csv'

In [7]:
data_df = pd.read_csv(file_path)
omni_df = pd.read_csv(omni_path)
Date = np.array(data_df['Date']).reshape(-1,shape)[:,0]
Time = np.array(data_df['Hours']).reshape(-1,shape)[:,0]
Mask = np.array(data_df['EXP Density']).reshape(-1,shape) == 0.0

Date_index = omni_df['Date'].values
Time_index = omni_df['Time'].values
IMF = omni_df['IMF'].values
speed = omni_df['Speed'].values

if time_bin == 1:
    Date_index = Date_index.reshape(-1,2)[:,0]
    Time_index = Time_index.reshape(-1,2)[:,0]
    IMF = IMF.reshape(-1,2)[:,0]
    speed = speed.reshape(-1,2)[:,0]

IMF_nan = np.isnan(IMF)
speed_nan = np.isnan(speed)

IMF_series = pd.Series(IMF)
speed_series = pd.Series(speed)

IMF = np.array(IMF_series.interpolate())
speed = np.array(speed_series.interpolate())

next_IMF = np.roll(IMF, -1)
next_speed = np.roll(speed, -1)

if time_bin == 1:
    speed_before = np.roll(speed, 1)
    dspeed_before = np.roll(speed, 2) - np.roll(speed, 1)
else: 
    speed_before = np.roll(speed, -2)
    dspeed_before = np.roll(speed, 4) - np.roll(speed, 2)

dIMF = next_IMF-IMF
dspeed = next_speed-speed

IMF_list = []
dIMF_list = []
speed_list = []
speed_before_list = []
dspeed_list = []
dspeed_before_list = []

for date in Date:
    date_index_mask = Date_index == date

    IMF_list.append(np.tile(IMF[date_index_mask], int(40/H_bin)))
    dIMF_list.append(np.tile(dIMF[date_index_mask], int(40/H_bin)))
    speed_list.append(np.tile(speed[date_index_mask], int(40/H_bin)))
    speed_before_list.append(np.tile(speed_before[date_index_mask], int(40/H_bin)))
    dspeed_list.append(np.tile(dspeed[date_index_mask], int(40/H_bin)))
    dspeed_before_list.append(np.tile(dspeed_before[date_index_mask], int(40/H_bin)))


IMF_array = np.array(IMF_list)
dIMF_array = np.array(dIMF_list)
speed_array = np.array(speed_list)
speed_before_array = np.array(speed_before_list)
dspeed_array = np.array(dspeed_list)
dspeed_before_array = np.array(dspeed_before_list)

IMF_array[Mask] = 0
dIMF_array[Mask] = 0
speed_array[Mask] = 0
speed_before_array[Mask] = 0
dspeed_array[Mask] = 0
dspeed_before_array[Mask] = 0

IMF_array = IMF_array.reshape(1, -1)[0]
dIMF_array = dIMF_array.reshape(1, -1)[0]
speed_array = speed_array.reshape(1, -1)[0]
speed_before_array = speed_before_array.reshape(1, -1)[0]
dspeed_array = dspeed_array.reshape(1, -1)[0]
dspeed_before_array = dspeed_before_array.reshape(1, -1)[0]

data_df['IMF'] = (IMF_array*100).astype(int)/100
data_df['IMF Gradient'] = (dIMF_array*100).astype(int)/100
data_df['Speed'] = (speed_array*100).astype(int)/100
data_df['Speed 1 hour before'] = (speed_before_array*100).astype(int)/100
data_df['Speed Gradient'] = (dspeed_array*100).astype(int)/100
data_df['Speed Gradient 1 hour before'] = (dspeed_before_array*100).astype(int)/100

data_df.to_csv(file_path, index=False)